In [1]:
import gmaps
import pandas as pd
import gmaps.datasets
import gmaps.geojson_geometries
from matplotlib.cm import viridis, plasma
from matplotlib.colors import to_hex

import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

# Census & gmaps API Keys
from config import census_key, g_key

# Configure gmaps
gmaps.configure(api_key=g_key)
states_geojson = gmaps.geojson_geometries.load_geometry('us-states')
# print(len(states_geojson['features']))
# print(states_geojson['features'][0])

In [2]:
# US cities
cities_data = pd.read_csv('resources/uscities_data.csv')
cities_geo = cities_data[['Lat', 'Lng']]
chicago = cities_data[cities_data['City']=='Chicago city, Illinois']
chicago_locations = chicago[['Lat', 'Lng']]
chicago_name = chicago['City'].values[0]
# US states
data = pd.read_csv('resources/states_2011.csv')
data_dict = data.filter(['State_Name', 'Crime_Adj'])
data_dict = data_dict.set_index('State_Name').to_dict()["Crime_Adj"]
# Extracting info for max value
rate_max = data[data["Crime_Adj"] == data["Crime_Adj"].max()]
marker_locations_max = rate_max[['Lat', 'Lng']]
rate_max_value = round(rate_max["Crime_Adj"],2).values[0]
# Extracting info for min value
rate_min = data[data["Crime_Adj"] == data["Crime_Adj"].min()]
marker_locations_min = rate_min[['Lat', 'Lng']]
rate_min_value = round(rate_min["Crime_Adj"],2).values[0]
il_rate = round(data_dict['Illinois'],2)

In [3]:
# We will need to scale the states values to lie between 0 and 1
min_state = min(data_dict.values())
max_state = max(data_dict.values())
state_range = max_state - min_state

def calculate_color(state):
    """
    Convert the state value to a color
    """
    # make state a number between 0 and 1
    normalized_state = (state - min_state) / state_range

    # invert state so that high value gives dark color
    inverse_state = 1.0 - normalized_state

    # transform the state value to a matplotlib color
    mpl_color = plasma(inverse_state)

    # transform from a matplotlib color to a valid CSS color
    gmaps_color = to_hex(mpl_color, keep_alpha=False)

    return gmaps_color

colors = []
for feature in states_geojson['features']:
    geo_state_name = feature['properties']['NAME']
    try:
        state = data_dict[geo_state_name]
        color = calculate_color(state)
    except KeyError:
        # no value for that state: return default color
        color = (0, 0, 0, 0.3)
    colors.append(color)

In [4]:
# 
fig = gmaps.figure()
states_layer = gmaps.geojson_layer(
    states_geojson,
    fill_color=colors,
    stroke_color=colors,
    fill_opacity=0.8)
fig.add_layer(states_layer)
cities_layer = gmaps.symbol_layer(cities_geo, fill_color='blue', stroke_color='blue', scale=1)
fig.add_layer(cities_layer)
marker_max = gmaps.marker_layer(marker_locations_max,
    info_box_content='Max Crime Rate per 100k: {}'.format(rate_max_value))
fig.add_layer(marker_max)
marker_min = gmaps.marker_layer(marker_locations_min,
    info_box_content='Min Crime Rate per 100k: {}'.format(rate_min_value))
fig.add_layer(marker_min)
marker_chi = gmaps.marker_layer(chicago_locations,
    info_box_content= 'Chicago,IL Rate per 100k: {}'.format(il_rate))
fig.add_layer(marker_chi)
fig

Figure(layout=FigureLayout(height='420px'))